# **Convolution Neural Network**

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

**Device**

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

**Hyper-parameter**

In [4]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

**Dataset**

In [5]:
train_dataset = torchvision.datasets.MNIST(root="./data/",
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root="./data/",
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=True)

**DataLoader**

In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

**CNN**

In [7]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

**Loss and Optimizer**

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

**Train Model**

In [9]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1265
Epoch [1/5], Step [200/600], Loss: 0.0753
Epoch [1/5], Step [300/600], Loss: 0.0246
Epoch [1/5], Step [400/600], Loss: 0.1623
Epoch [1/5], Step [500/600], Loss: 0.0278
Epoch [1/5], Step [600/600], Loss: 0.0651
Epoch [2/5], Step [100/600], Loss: 0.0561
Epoch [2/5], Step [200/600], Loss: 0.0355
Epoch [2/5], Step [300/600], Loss: 0.1543
Epoch [2/5], Step [400/600], Loss: 0.0077
Epoch [2/5], Step [500/600], Loss: 0.0236
Epoch [2/5], Step [600/600], Loss: 0.0221
Epoch [3/5], Step [100/600], Loss: 0.0736
Epoch [3/5], Step [200/600], Loss: 0.0408
Epoch [3/5], Step [300/600], Loss: 0.0548
Epoch [3/5], Step [400/600], Loss: 0.0225
Epoch [3/5], Step [500/600], Loss: 0.0114
Epoch [3/5], Step [600/600], Loss: 0.1016
Epoch [4/5], Step [100/600], Loss: 0.0053
Epoch [4/5], Step [200/600], Loss: 0.0073
Epoch [4/5], Step [300/600], Loss: 0.0993
Epoch [4/5], Step [400/600], Loss: 0.0131
Epoch [4/5], Step [500/600], Loss: 0.0914
Epoch [4/5], Step [600/600], Loss:

**Test Model**

In [11]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

**Save & Load Model**

In [12]:
# Save the model checkpoint
torch.save(model.state_dict(), './model/CNN_model.ckpt')

In [15]:
model.load_state_dict(torch.load('./model/CNN_model.ckpt'))

<All keys matched successfully>